In [21]:
import pandas as pd
import altair as alt
import numpy as np

In [52]:
data = pd.read_csv("./partos-e-cesarianas.csv",sep=';')
data["ano"] = data["Período"].str[0:4].astype(int)
data["mês"] = data["Período"].str[5:7].astype(int)

In [53]:
list_of_hospitals = data["Instituição"].unique()

all_hospitals = []
 
for hospital in (list_of_hospitals):
    starting_month = ""
    if (hospital == "Hospital de Vila Franca de Xira, EPE"):
        starting_month = 6
    elif ((hospital == "Hospital de Braga, EPE") |( hospital == "Hospital Santa Maria Maior, EPE")):
        starting_month = 9
    else:
        starting_month = 1
    data_hospital = data[data["Instituição"] == hospital]
    data_hospital =  data_hospital.sort_values(by=['ano', 'mês'])
    data_hospital["partos_mês"] = np.where((data_hospital["mês"]) == starting_month, data_hospital["Nº Total de Partos"], data_hospital["Nº Total de Partos"].diff())
    data_hospital["cesarianas_mês"] = np.where((data_hospital["mês"]) == starting_month, data_hospital["Nº Cesarianas"], data_hospital["Nº Cesarianas"].diff())
    data_hospital['percent_cesarianas_mês'] = (data_hospital["cesarianas_mês"]/data_hospital["partos_mês"] *100).round(1) 
    data_hospital["percent_cesarianas_ano"] = np.where((data_hospital["mês"]) == 12, (data_hospital["Nº Cesarianas"]/data_hospital["Nº Total de Partos"] *100).round(1), "não aplicável"  )
    all_hospitals.append(data_hospital)

final_hospitals = pd.concat(all_hospitals)
final_hospitals = concat_hospitals[concat_hospitals["percent_cesarianas_mês"] > 0] 
final_hospitals = concat_hospitals[concat_hospitals["percent_cesarianas_mês"] < 101] 


final_hospitals.rename(columns={"Período": "período", 
                     "Região": "região", 
                     "Instituição": "instituição", 
                     "Localização Geográfica":"localização_geográfica",
                     "Nº Total de Partos": "total_partos",
                     "Nº Cesarianas": "total_cesarianas"
                    }, inplace=True)
final_hospitals.to_csv("partos_hospitais_publicos.csv")



In [51]:
selection = alt.selection_multi(fields=['Instituição'], bind='legend')

all_ = alt.Chart(final_hospitals).mark_tick(color="gray").encode(
    x=alt.X('período:O'),
    y=alt.Y('percent_cesarianas_ano:Q', scale=alt.Scale(zero=False)),
    color = alt.Color('instituição:N', scale={"range": ["lightgray"]}),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
).add_selection(
    selection
)

mean =alt.Chart(final_hospitals).mark_tick(color="red").encode(
    x='período:O',
    y='mean(percent_cesarianas_ano):Q',
    tooltip="mean(percent_cesarianas_ano):Q"

)
all_ + mean

alt.LayerChart(...)